In [1]:
import torch
import torch.nn as nn
import tqdm
import json
import os
from datasets import load_dataset
from transformers.models.opt.modeling_opt import (
    OPTAttention,
    OPTDecoderLayer,
    OPTForCausalLM,
)
from transformers import GPT2Tokenizer, AutoTokenizer, AutoModelForCausalLM
from smoothquant.smooth import smooth_lm
from smoothquant.fake_quant_2_bits import quantize_qwen2 as quantize_qwen2_2
from smoothquant.fake_quant_4_bits import quantize_qwen2 as quantize_qwen2_4
from smoothquant.fake_quant_6_bits import quantize_qwen2 as quantize_qwen2_6
from smoothquant.fake_quant_8_bits import quantize_qwen2 as quantize_qwen2_8

SyntaxError: invalid syntax (280961585.py, line 2)

In [2]:
class Evaluator:
    def __init__(self, dataset, tokenizer, device):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.device = device

        # tokenize the dataset
        def tokenize_function(examples):
            example = self.tokenizer(examples["text"])
            return example

        self.dataset = self.dataset.map(tokenize_function, batched=True)
        self.dataset.set_format(type="torch", columns=["input_ids"])

    @torch.no_grad()
    def evaluate(self, model):
        model.eval()
        # The task is to predict the last word of the input.
        total, hit = 0, 0
        for batch in self.dataset:
            input_ids = batch["input_ids"].to(self.device).unsqueeze(0)
            label = input_ids[:, -1]
            outputs = model(input_ids)
            
            # Check sequence length to avoid IndexError
            sequence_length = outputs.logits.size(1)
            if sequence_length < 2:
                # If the sequence length is less than 2, use the last token
                last_token_logits = outputs.logits[:, -1, :]
            else:
                # Otherwise, use the second-to-last token
                last_token_logits = outputs.logits[:, -2, :]
            
            pred = last_token_logits.argmax(dim=-1)
            total += label.size(0)
            hit += (pred == label).sum().item()
        acc = hit / total
        return acc

In [15]:
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")
# dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
# dataset = dataset.select(range(1000))  # 데이터셋에서 처음 1000개를 선택
dataset = load_dataset("lambada", split="validation[:1000]")  # LAMBADA 데이터셋
evaluator = Evaluator(dataset, tokenizer, "cuda")


Map: 100%|██████████| 1000/1000 [00:00<00:00, 8337.85 examples/s]


In [19]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B", torch_dtype=torch.float16, device_map="auto"
).to("cuda:0")

evaluator = Evaluator(dataset, tokenizer, "cuda:0")


Map: 100%|██████████| 1000/1000 [00:00<00:00, 9323.64 examples/s]


In [20]:
opt_model = evaluator.evaluate(model)
print(f"Qwen2.5 0.5B model accuracy: {opt_model}")

Qwen2.5 0.5B model accuracy: 0.666


In [29]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B", torch_dtype=torch.float16, device_map="auto"
)

act_scales = torch.load("act_scales/Qwen2.5-0.5b.pt")
smooth_lm(model, act_scales, 0.85)
model_smoothquant = quantize_qwen2_2(model).to("cuda:0") 

/tmp/ipykernel_46272/1321960318.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  act_scales = torch.load("act_scales/Qwen2.5-0.5b.pt")


In [30]:
acc_smoothquant = evaluator.evaluate(model_smoothquant)
print(f"SmoothQuant W8A8 quantized model accuracy: {acc_smoothquant}")

SmoothQuant W8A8 quantized model accuracy: 0.0
